<a href="https://colab.research.google.com/github/mjkellykim/GANinAction/blob/main/ch3_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels) # 입력 이미지 차원

z_dim = 100 # 생성자의 입력으로 사용될 잡음 벡터의 크기

In [3]:
# Generator
def build_generator(img_shape, z_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim)) # 완전 연결층
    model.add(LeakyReLU(alpha=0.01)) # LeakyReLU 활성화 함수
    model.add(Dense(28*28*1, activation='tanh')) # tanh 활성화 함수를 사용한 출력층
    model.add(Reshape(img_shape)) # 생성자 출력을 이미지 차원으로 변경
    return model

In [4]:
# Discriminator
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape)) # 입력 이미지 일렬로 펼치기
    model.add(Dense(128)) # 완전 연결층
    model.add(LeakyReLU(alpha=0.01)) # LeakyReLU 활성화 함수
    model.add(Dense(1,activation='sigmoid')) # sigmoid 활성화 함수를 사용한 출력층
    return model

In [5]:
# Build the GAN Model
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [6]:
discriminator = build_discriminator(img_shape) # 판별자 모델 만들고 컴파일 하기
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

generator = build_generator(img_shape, z_dim) # 생성자 모델 만들기

discriminator.trainable = False # 생성자 훈련시에 판별자 파라미터 동결
 
gan = build_gan(generator, discriminator) # 생성자를 훈련하기 위해 동결한 판별자로 GAN 모델 만들고 컴파일하기
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [7]:
# Training
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    
    (X_train, _), (_, _) = mnist.load_data() # 데이터셋 로드

    X_train = X_train / 127.5 - 1.0 # [0, 255] 흑백 픽셀값을 [-1, 1] 사이로 스케일 조정
    X_train = np.expand_dims(X_train, axis=3)

    real = np.ones((batch_size, 1)) # 진짜 이미지 레이블 : 모두 1
    fake = np.zeros((batch_size, 1)) # 가짜 이미지 레이블 : 모두 0

    for iteration in range(iterations):
        idx = np.random.randint(0, X_train.shape[0], batch_size) # 진짜 이미지에서 랜덤 배치 가져오기
        imgs = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, 100)) # 가짜 이미지 배치 생성
        gen_imgs = generator.predict(z) ## kmjeung

        d_loss_real = discriminator.train_on_batch(imgs, real) # 판별자 훈련
        d_loss_fake = discriminator.train_on_batch(gen_imgs ,fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

        z = np.random.normal(0, 1, (batch_size, 100)) # 가짜 이미지 배치 생성
        gen_imgs = generator.predict(z)

        g_loss = gan.train_on_batch(z, real) # 생성자 훈련

        if (iteration + 1) % sample_interval == 0:
            losses.append((d_loss,g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)
            print("%d [D 손실: %f, 정확도: %.2f%%] [G 손실: %f]" % # 훈련 과정 출력
                  (iteration+1, d_loss, 100.0 * accuracy, g_loss))
            
        sample_images(generator)

In [8]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    gen_imgs = generator.predict(z) # 랜덤한 잡음에서 이미지 생성

    gen_imgs = 0.5 * gen_imgs + 0.5 # 이미지 픽셀 값을 [0,1] 범위로 스케일 조정

    fig, axs = plt.subplots(image_grid_rows, # 이미지 그리드 설정
                            image_grid_columns, 
                            figsize=(4,4),
                            sharey=True,
                            sharex=True)
    
    cnt = 0
    for i in range(image_grid_columns):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray') # 이미지 그리드 출력
            axs[i, j].axis('off')
            cnt += 1

In [ ]:
iterations = 20000 # 하이퍼파라미터 설정
batch_size = 128
sample_interval = 1000

train(iterations, batch_size, sample_interval) # 지정된 횟수 동안 GAN 훈련

11501568/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':
